Initializing staff

In [1]:
import numpy as np
import scipy.integrate as integrate
import lhapdf

Setting up constants

In [2]:
s_h = 27**2 # in GeV (TeV)
p_t = [1,2,3,4]#[0.25,0.50,0.75,1.00,1.25,1.50,1.75,2.00,2.25,2.50,2.75,3.00,3.25,3.50,3.75,4.00] #GeV
i_ = [-5,-4,-3,-2,-1,1,2,3,4,5]
alpha_em=1./137
const = 1/(s_h*8*(np.pi**2))
#nNNPDF20_nlo_as_0118_D2
dsigma = [0] * 6
#result = 0.
sigma_dp = []

Setting up pdf sets

In [3]:
pdfs = [lhapdf.mkPDF("NNPDF23_nlo_as_0118", 0), lhapdf.mkPDF("nNNPDF20_nlo_as_0118_D2", 0), lhapdf.mkPDF("cteq6l1", 0)]

LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/NNPDF23_nlo_as_0118/NNPDF23_nlo_as_0118_0000.dat
NNPDF23_nlo_as_0118 PDF set, member #0, version 6; LHAPDF ID = 229800
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/nNNPDF20_nlo_as_0118_D2/nNNPDF20_nlo_as_0118_D2_0000.dat
nNNPDF20_nlo_as_0118_D2 PDF set, member #0, version 1; LHAPDF ID = 30010300
LHAPDF 6.5.4 loading /home/yoren/bnl/lhapdf/LHAPDF-6.5.4/install/share/LHAPDF/cteq6l1/cteq6l1_0000.dat
cteq6l1 PDF set, member #0, version 4; LHAPDF ID = 10042


Function for Compton and annihilation cross-section

In [10]:
def compton_anih_crossection(y_1, y_2, p, i, iset):
    #print("y_1", y_1, y_2, s_h)
    if abs(y_1)>2 or y_2>2: return 0.
    x_1 = (2*p/np.sqrt(s_h))* np.exp((y_1 + y_2)/2)*np.cosh(abs((y_1-y_2)/2))
    x_2 = (2*p/np.sqrt(s_h))* np.exp(-(y_1 + y_2/2))*np.cosh(abs((y_1-y_2)/2))
    s = x_1*x_2*s_h
    #print(x_1, x_2, p, s_h, s, 1. - 4*(p)**2/s)
    if np.all( [x_1 > 0 , x_1 < 1 , x_2 > 0 , x_2 < 1, 1. - 4*(p)**2/s > 0] ):
        cos_theta = np.sqrt(1 - 4*(p)**2/s)
        u = (-1*s/2)*(1 + cos_theta)
        t = (-1*s/2)*(1 - cos_theta)
        Q2 = -t
        #if Q2<1: print("wrong q2")
        pdf = pdfs[iset]
        alfa_s = pdf.alphasQ2(Q2)
        pdf_i = max(pdf.xfxQ2(i, x_1, Q2)/x_1,0.)
        pdf_j = max(pdf.xfxQ2(21, x_2, Q2)/x_2,0.)
        #print(Q2,pdf.xfxQ2(i, x_1, Q2),x_1)
        pdf_i2 = max(pdf.xfxQ2(i, x_2, Q2)/x_2,0)
        pdf_j2 = max(pdf.xfxQ2(21, x_1, Q2)/x_1,0)
        sigma_compron_dt = -(8*np.pi*alfa_s*alpha_em/s)*((s**2 + u**2)/(4*u*s))
        #sigma_part = x_1*x_2*(pdf_i*pdf_j*sigma_dt_1 + pdf_i2*pdf_j2*sigma_dt_2)
        sigma = pdf_i*pdf_j*sigma_dt+pdf_i2*pdf_j2*sigma_dt
        #print(s_h, s, cos_theta, u, t, 1 - 4*(p)**2/s, sigma)
        if sigma<0: print(u,pdf_i,pdf_j )
        
        return  sigma
    return 0.

In [11]:
print(compton_anih_crossection(0.1,0.6,1,1,0))

2.5463618463241815


Cross-section integration

Integration Bounds and accuracy

In [12]:
def bounds_y(p,i,iset):
    return [np.log(p/np.sqrt(s_h)), np.log(np.sqrt(s_h)/p)]
        
def bounds_x(y_1,p,i,ist):
    return [np.log(p/(np.sqrt(s_h) - p*np.exp(-1 * y_1))), np.log((np.sqrt(s_h) - p*np.exp(-y_1))/p)]
    
def opts0(y_1,p,i,iset):
    return {'epsrel' : 0.05}

def opts1(p,i,iset):
    return {'epsrel' : 0.05}

Integration function

In [13]:
def integration_cs(pT, quarks, iset):
    result=[]
    for p in pT:
        result.append(0)
        for i in quarks:     
                
            integral = integrate.nquad(compton_anih_crossection, [bounds_x, bounds_y], args=(p,i,iset), opts=[opts0,opts1])
            #print(p,i,integral)
        
            result[-1] += integral[0]
    return result

Parallel calculating for $p+p$

In [14]:
import multiprocess as mp

Ntr = 4
pool = mp.Pool(Ntr)

hist_array = pool.starmap(integration_cs, [([1.0*i+1.0], i_, 0) for i in range(Ntr)])
    
pool.close()   

1.0 -5 (0.0, 0)2.0
 -5 (0.0, 0)
3.0 -5 (3.006555190347918e-11, 2.1340684159368843e-10)
1.0 -4 (3.8506839800399133e-17, 3.7729189176168586e-17)
4.0 -5 (6.04932078503469e-08, 1.4878190844999177e-08)
3.0 -4 (4.487874231897093e-05, 2.671294770631864e-06)
2.0 -4 (0.0013459112379767568, 6.1936656425006e-05)
1.0 -3 (0.4290948950944508, 0.01569238213523716)
4.0 -4 (1.9794842270814646e-06, 1.2881872920157632e-07)
3.0 -3 (0.00011071459609886212, 6.608188480122882e-06)
1.0 -2 (1.2440458126738876, 0.04399266650292366)
2.0 -3 (0.00518941728531763, 0.000251118215433563)
2.0 -2 (0.013739957768402781, 0.0005351600693121753)
4.0 -3 (2.5334041866101186e-06, 2.0896347763966218e-07)
1.0 -1 (1.7207368052442367, 0.08068061659859425)
3.0 -2 (0.00039945792217672505, 2.0023187571082257e-05)
2.0 -1 (0.019048808752319006, 0.0009202064795662845)
4.0 -2 (1.922155082842371e-05, 1.2836249334797717e-06)
1.0 1 (5.616345603160603, 0.21128569688540771)
3.0 -1 (0.00046942089328256723, 2.5658405889957608e-05)
4.0 -1 (1.62

Parallel calculating for $d+d$

In [15]:
pool_dd = mp.Pool(Ntr)

hist_array_dd = pool_dd.starmap(integration_cs, [([1.0*i+1.0], i_, 1) for i in range(Ntr)])
    
pool.close()   

3.02.01.0  -5 -5-5   (0.0, 0)(0.0, 0)(0.0, 0)


1.0 -4 (0.0, 0)
4.0 -5 (2.4866371610059994e-08, 1.4813319651105466e-08)
3.0 -4 (3.809382114452951e-05, 2.1036257809925663e-06)
1.0 -3 (1.116674077223018, 0.04115227461906977)
2.0 -4 (0.0010352695639440504, 4.854237312277893e-05)
4.0 -4 (1.897429916423381e-06, 1.335075405719574e-07)
3.0 -3 (0.00038370324756938215, 1.6562200294571264e-05)
1.0 -2 (1.6527167732760448, 0.07187313730552479)
4.0 -3 (2.2644549920355457e-05, 1.2153052088970467e-06)
2.0 -3 (0.011498850784646507, 0.0004052328555092539)
4.0 -2 (2.1710944048188205e-05, 1.0234636474930845e-06)
1.0 -1 (1.6527167732760448, 0.07187313730552479)
3.0 -2 (0.0003853254484503791, 1.8793503752811232e-05)
4.0 -1 (2.1710944048188205e-05, 1.0234636474930845e-06)
2.0 -2 (0.013623315805826725, 0.0004661515816343168)
1.0 1 (8.356757831024067, 0.21794269300878177)
4.0 1 (0.0005320092011623957, 2.712057795124634e-05)
3.0 -1 (0.0003853254484503791, 1.8793503752811232e-05)
4.0 2 (0.0005320092011623957, 2

Factors

In [18]:
#print(hist_array,hist_array_dd)
res=[]
for ival in range(len(hist_array)):
    res.append(hist_array_dd[ival][0]/(hist_array[ival][0]+1e-20))
print("pT:1.0,2.0,3.0,4.0",res)

pT:1.0,2.0,3.0,4.0 [1.2152097252484704, 0.9510251849251178, 0.9517870004622941, 0.9574126587160509]
